# 1b. Pull General User Tweets with API

In [9]:
import tweepy, json, logging
import pandas as pd
import numpy as np
import matplotlib as mlt
import pickle

In [10]:
logging.basicConfig(filename='twitter_error_log_1.log',filemode='w', level=logging.ERROR)

In [11]:
consumer_key = 'XXXXXXXXX'
consumer_secret = 'XXXXXXXXX'

access_token = 'XXXXXXXXX'
access_token_secret = 'XXXXXXXXX'

In [12]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,
                 parser=tweepy.parsers.JSONParser()
                )

In [13]:
virginia_search = ('(virginia AND (democrats OR republicans OR “house of delegates” OR hod OR #GOTV OR politics OR elections OR governor OR senate)) OR #VAGov OR #VASen OR #VA2021 OR terrymcauliffe OR (terry AND mcauliffe) OR glennyoungkin OR (glenn AND youngkin)')

nj_search = ('((“new jersey” OR NJ) AND (democrats OR republicans OR house OR #GOTV OR politics OR elections OR governor OR senate)) OR #NJGov OR #NJSen OR #NJ2021 OR philmurphy OR (phil AND murphy) OR jackciattarelli OR (jack AND ciattarelli)')


In [14]:
def pull_public_tweets(searchQuery, state):
    maxTweets = 2000000000 # Some arbitrary large number
    tweetsPerQry = 100  # max the API permits

    # default to no lower limit, go as far back as API allows
    sinceId = None

    # default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1

    tweetCount = 0
    tweetFiles = 0
    msg =  "Downloading max {0} tweets".format(maxTweets)
    logging.error(msg)
    tweets = []
    while 4000*tweetFiles + tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode = 'extended')
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId, tweet_mode = 'extended')
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1), tweet_mode = 'extended')
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId, tweet_mode = 'extended')

            new_tweets = new_tweets['statuses']
            if len(new_tweets) == 0:
                msg = 'No tweets found'
                logging.error(msg)
                break

            tweets.extend(new_tweets)
            tweetCount += len(new_tweets)
            msg = "Downloaded {0} tweets".format(tweetCount + tweetFiles*4000)
            logging.error(msg)
            max_id = new_tweets[-1]['id']

            if tweetCount > 4000:
                with open(state+'tweets'+str(tweetFiles)+'.json', 'w') as outfile:  
                    json.dump(tweets, outfile)
                msg = "JSON file saved"
                logging.error(msg)
                tweetFiles += 1
                tweetCount = 0
                del tweets[:]

        except tweepy.TweepError as e:
            msg = 'Query failed when max_id equaled {0}: {1}'.format(max_id, e)
            logging.error(msg)

    if tweetCount > 0:
        with open(state+'tweets'+str(tweetFiles)+'.json', 'w') as outfile:  
            json.dump(tweets, outfile)
        msg = "JSON file saved"
        logging.error(msg)
        tweetFiles += 1
        tweetCount = 0

In [15]:
pull_public_tweets(virginia_search, state = 'virginia')

In [16]:
pull_public_tweets(nj_search, state = 'new_jersey')